In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
from datetime import date
from dateutil.relativedelta import relativedelta

#Stock screaner with simple trading startegy and backtesting algorithm



In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tabels = pd.read_html(url)
sp500 = tabels[0]

sp500_tickers = sp500['Symbol'].tolist()
sp500_tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 

In [ ]:

end = date.today()

start = end - relativedelta(years=5)

def bollinger_bands(df,window=20):
    df[f'SMA{window}'] = df['Close'].rolling(window=window).mean()
    df[f'STD{window}'] = df['Close'].rolling(window=window).std()

    df[f'UpperBand {window}'] = df[f'SMA{window}'] + (df[f'STD{window}'] * 2)
    df[f'LowerBand {window}'] = df[f'SMA{window}'] - (df[f'STD{window}'] * 2)
    return df

def create_RSI(df,window=14):
    delta = df['Close'].diff()

    gain = (delta.where(delta > 0, 0))
    loss = (-delta.where(delta < 0, 0))

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    RS = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + RS))
    return df


def money_flow_index(df,window=14):
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    raw_money_flow = typical_price * df['Volume']

    possitive_flow = (raw_money_flow.where(typical_price > typical_price.shift(1), 0))
    negative_flow = (raw_money_flow.where(typical_price < typical_price.shift(1), 0))

    window=14
    possitive_flow_sum = possitive_flow.rolling(window=window).sum()
    negative_flow_sum = negative_flow.rolling(window=window).sum()
    money_flow_ratio = possitive_flow_sum / negative_flow_sum

    df['MFI'] = 100 - (100 / (1 + money_flow_ratio))
    return df


def algorithm(ticker,start=start,end=end,interval='1d'):
    df = yf.download(ticker,start=start,end=end,interval=interval)
    df.columns = [col[0] for col in df.columns]

    #calc EMA and BB
    df = bollinger_bands(df,window=12)
    df = bollinger_bands(df,window=20)
    df = bollinger_bands(df,window=50)

    #calc RSI for momentum
    df = create_RSI(df,window=14)

    #calc MFI for momentum
    df = money_flow_index(df,window=14)

    #Create logic for buying and selling signals
    low= df['Low']
    high = df['High']
    sma12 = df['SMA12']
    sma20 = df['SMA20']
    sma50 = df['SMA50']

    bollinger_upper_20 = df['UpperBand 20']
    bollinger_lower_20 = df['LowerBand 20']

    bollinger_upper_50 = df['UpperBand 50']
    bollinger_lower_50 = df['LowerBand 50']

    rsi = df['RSI']
    mfi = df['MFI']

    #initialize the signals
    buy_signals = []
    sell_signals = []
    action = []

    for i in range(0,len(df)):
        low_val = low[i]
        high_val = high[i]
        sma12_val = sma12[i]
        sma20_val = sma20[i]
        sma50_val = sma50[i]
        bollinger_lower_val_20 = bollinger_lower_20[i]
        bollinger_upper_val_20 = bollinger_upper_20[i]
        bollinger_lower_val_50 = bollinger_lower_50[i]
        bollinger_upper_val_50 = bollinger_upper_50[i]
        rsi_val = rsi[i]
        mfi_val = mfi[i]

        buy_condition = (low_val < bollinger_lower_val_20) and rsi_val < 30 and mfi_val < 25

        sell_condition = (high_val > bollinger_upper_val_20) and rsi_val > 70 and mfi_val > 75

        if buy_condition:
            buy_signals.append(low_val*0.95)
            sell_signals.append(np.nan)
            action.append('Buy')
        elif sell_condition:
            buy_signals.append(np.nan)
            sell_signals.append(high_val*1.05)
            action.append('Sell')
        else:
            buy_signals.append(np.nan)
            sell_signals.append(np.nan)
            action.append('Hold')

    df['buy_signals'] = buy_signals
    df['sell_signals'] = sell_signals
    df['action'] = action

    initial_investment = 10000
    cash = initial_investment
    position = 0
    portfolio_val = []

    share_cost = df['Open'].iloc[0]
    num_shares = cash / share_cost
    df['Buy_Hold_Value'] = df['Close']*num_shares


    for i in range(0,len(df)):
        action = df['action'].iloc[i]
        close_val = df['Close'].iloc[i]

        if action == 'Buy' and cash > 0:
            position = cash/close_val
            cash = 0
        elif action == 'Sell' and position > 0:
            cash = position*close_val
            position = 0
        elif action == 'Hold':
            pass

        portfolio_val.append(cash + position*close_val)

    df['Portfolio_Value'] = portfolio_val
    return df

In [ ]:
df = algorithm('NVDA',start=start,end=end,interval='1d')
df

In [6]:
action_count = df['action'].value_counts()
action_count

action
Hold    1205
Sell      42
Buy        9
Name: count, dtype: int64

In [8]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,row_heights=[0.6,0.2,0.2], subplot_titles=('Stock Price with Buy/Sell Signals','RSI (13)','MFI (14)'))

fig.add_trace(go.Candlestick(x=df.index,open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'],name='Candlestick'))

fig.add_trace(go.Scatter(
    x = df.index,
    y = df['LowerBand 20'],
    mode = 'lines',
    name='Lower Band 20',
    line=dict(color='blue', width=1,dash='dash')
))

fig.add_trace(go.Scatter(
    x = df.index,
    y = df['UpperBand 50'],
    mode = 'lines',
    name='Upper Band 50',
    line=dict(color='red', width=1,dash='dash')
))

fig.add_trace(go.Scatter(
    x = df.index,
    y = df['LowerBand 50'],
    mode = 'lines',
    name='Lower Band 50',
    line=dict(color='green', width=1,dash='dash')
))

fig.add_trace(go.Scatter(x=df.index,y=df['buy_signals'],mode='markers',marker=dict(color='green',symbol='triangle-up',size=10),name='Buy Signal'))

fig.add_trace(go.Scatter(x=df.index,y=df['sell_signals'],mode='markers',marker=dict(color='red',symbol='triangle-down',size=10),name='Sell Signal'))

fig.add_trace(go.Scatter(x=df.index,y=df['RSI'],name='RSI',line=dict(color='purple', width=2)),row=2,col=1)

fig.add_hline(y=70,line=dict(color='red',width=1,dash='dash'),row=2,col=1)
fig.add_hline(y=30,line=dict(color='green',width=1,dash='dash'),row=2,col=1)

fig.add_trace(go.Scatter(x=df.index,y=df['MFI'],name='MFI',line=dict(color='brown', width=2)),row=3,col=1)

fig.add_hline(y=80,line=dict(color='red',width=1,dash='dash'),row=3,col=1)
fig.add_hline(y=20,line=dict(color='green',width=1,dash='dash'),row=3,col=1)

fig.update_layout(xaxis_rangeslider_visible=False,template='plotly_dark')

fig.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
result_dic = {}
for ticker in sp500_tickers:
    try:
        ticker = ticker.replace('.','-')
        #print(f'>>Now running: {ticker}')
        df = algorithm(ticker,start=start,end=end,interval='1d')

        stock = yf.Ticker(ticker)

        analysis = stock.recommendations
        analysis = analysis[(analysis['period']=='0m')]

        count_buy = analysis['strongBuy']+analysis['buy']
        count_buy = count_buy[0]
        count_sell = analysis['strongSell']+analysis['sell']
        count_sell = count_sell[0]

        is_more_buys = count_buy > count_sell

        analysis['IsMoreBuys'] = ['Y' if a > b else 'N' for a,b in zip(analysis['strongBuy']+analysis['buy'],analysis['strongSell']+analysis['sell'])]

        result_dic[f'{ticker}'] = ([df['Buy_Hold_Value'].iloc[-1],df['Portfolio_Value'].iloc[-1],count_buy,count_sell,analysis['IsMoreBuys'].iloc[-1]])
    except Exception as e:
        print(e)
print(result_dic)

In [14]:
df_results = pd.DataFrame.from_dict(result_dic,orient='index',columns=['Buy_Hold_Value','Portfolio_Value','Count_Buy','Count_Sell','IsMoreBuys'])
df_results['Difference'] = df_results['Portfolio_Value'] - df_results['Buy_Hold_Value']
df_results['Buy_Hold_%'] = (df_results['Buy_Hold_Value'] - 10000) / 10000
df_results['Portfolio_%'] = (df_results['Portfolio_Value'] - 10000) / 10000
df_results['Difference_%'] = (df_results['Portfolio_%'] - df_results['Buy_Hold_%'])

df_buys = df_results[(df_results['IsMoreBuys']==True)&df_results['Difference']>0]
df_results.sort_values(by='Difference_%',ascending=False)

,Buy_Hold_Value,Portfolio_Value,Count_Buy,Count_Sell,IsMoreBuys,Difference,Buy_Hold_%,Portfolio_%,Difference_%
HAL,17266.900027,45856.346832,22,0,Y,28589.446805,0.726690,3.585635,2.858945
MHK,9459.934947,32114.206646,8,0,Y,22654.271699,-0.054007,2.211421,2.265427
SLB,16569.894838,37367.997846,26,0,Y,20798.103009,0.656989,2.736800,2.079810
LW,6153.972216,21510.590056,5,0,Y,15356.617841,-0.384603,1.151059,1.535662
EQR,11114.849314,23593.399672,10,0,Y,12478.550359,0.111485,1.359340,1.247855
...,...,...,...,...,...,...,...,...,...
AVGO,77894.303355,14015.578567,36,0,Y,-63878.724788,6.789430,0.401558,-6.387872
EQT,90022.784285,20310.924293,14,1,Y,-69711.859992,8.002278,1.031092,-6.971186
PLTR,84400.001526,4945.354796,4,5,N,-79454.646730,7.440000,-0.505465,-7.945465
SMCI,149161.578920,7669.895098,3,2,Y,-141491.683822,13.916158,-0.233010,-14.149168
